In [ ]:
from gensim import corpora
from gensim.models import LdaModel
import json
from data_analysis.OpeningLDA import OpeningLDA
from classes.QueryHandler import QueryHandler

## LDA

In [ ]:
#opening_lda = OpeningLDA(num_passes=5, print_debug=True) 

#print(len(opening_lda.texts.keys()))
#opening_lda.lda_model.print_topics(num_words=15)

#opening_lda.save('lda')


In [ ]:
opening_lda = OpeningLDA.load("lda")

In [ ]:
opening_lda.__construct_topic_doc_matrix__()

In [ ]:
query_handler = QueryHandler()
query_handler.opening_lda = opening_lda

sample_query1 = { 
                 "message": "capture the center, develop light square bishop",
                 "color": "white" 
}
query_handler.handle_user_query(sample_query1, debug=True)

## GPT

In [3]:
from data_analysis.OpeningGPT import OpeningGPT
from scraper.Paths import Paths

opening_gpt = OpeningGPT()


### Precomputing summaries for all ECOs 

In [4]:
opening_gpt.precompute_all_summaries(save_to=Paths.RAND_SUMMARIES_JSON, print_debug=True, temperature=0.8, min_sum_len=100, max_sum_len=300, overwrite_existing=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (13817 > 1024). Running this sequence through the model will result in indexing errors


OpeningGPT: precompute_all_summaries - Skipping ECO "B44" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "D12" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "B88" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "D15" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "B43" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "B81" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "B75" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "D23" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skipping ECO "D24" because it's summary has already been computed.
OpeningGPT: precompute_all_summaries - Skippin

Your max_length is set to 300, but your input_length is only 80. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)


	n = 3244
	num_chars = 3219
	len(tokenized chunk) = 1013 | 3228
	summary: "1.e4 e5 2.Nf3 Nc6 3.Bc4 Bc5 4.Nc3 Nf6 is a favourite among younger players due to its simple and easy development. Black can preserve the symmetry by 4...Bc5, leading to the quiet Giuoco Pianissimo. After 4...Nxe4, Black can perform the center fork trick by pseudo-sacrificing a knight with 4... Nxe4!. Then 5.Bxf7+?, though superficially attractive, relinquishes bishop pair and central control to Black."
	summary len = 122

OpeningGPT: __process_text__ - Iteration #2
	chars_processed = 6460/6820
	toks_processed = 2031/2141
	n = 361
	num_chars = 280
	len(tokenized chunk) = 78 | 281
	summary: "The Halloween Gambit is a chess move. It is one of the most popular chess moves of all time. The game is played in a number of different ways. It can be played in any number of ways, depending on the strategy used. For more information on the game, see: http://www.cnn.com/2013/01/30/chess/halloween-gambit.html#storylink=cpy. 

Your max_length is set to 300, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


...

Your max_length is set to 300, but your input_length is only 165. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=82)


	n = 3534
	num_chars = 3510
	len(tokenized chunk) = 1022 | 3524
	summary: "The Three Knights Variation of the Queen's Gambit Declined is usually reached from the move order 1.d4 Nf6 2.c4 e6 3.Nf3 d5 4.Nc3, played to avoid the Nimzo-Indian Defense. The Vienna Variation occurs after Black plays 4...dxc4 in the Three Knightsvariation. An important line in this variation is the Endgame Line where the game continues. This line has been played by top grandmasters such as Wesley So, Anish Giri, and Magnus Carlsen aiming for a draw."
	summary len = 120

OpeningGPT: __process_text__ - Iteration #3
	chars_processed = 10445/11174
	toks_processed = 3063/3262
	n = 729
	num_chars = 641
	len(tokenized chunk) = 163 | 641
	summary: "Placing the bishop on Bg5 allows Black to exchange more freely with moves like Nf6-e4. The move Bf4 is designed to restrict Black's opportunities in this way. Play usually continues with 5...0-0 6.e3 c5 7.dxc5 Bxc5.[19] Peter Leko, usually an e4 player, used this variation 

Your max_length is set to 300, but your input_length is only 165. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=82)


	n = 3534
	num_chars = 3510
	len(tokenized chunk) = 1022 | 3524
	summary: "The Three Knights Variation of the Queen's Gambit Declined is usually reached from the move order 1.d4 Nf6 2.c4 e6 3.Nf3 d5 4.Nc3, played to avoid the Nimzo-Indian Defense. The Vienna Variation occurs after Black plays 4...dxc4 in the Three Knightsvariation. An important line in this variation is the Endgame Line where the game continues. This line has been played by top grandmasters such as Wesley So, Anish Giri, and Magnus Carlsen aiming for a draw."
	summary len = 120

OpeningGPT: __process_text__ - Iteration #3
	chars_processed = 10445/11174
	toks_processed = 3063/3262
	n = 729
	num_chars = 641
	len(tokenized chunk) = 163 | 641
	summary: "Placing the bishop on Bg5 allows Black to exchange more freely with moves like Nf6-e4. The move Bf4 is designed to restrict Black's opportunities in this way. Play usually continues with 5...0-0 6.e3 c5 7.dxc5 Bxc5.[19] Peter Leko, usually an e4 player, used this variation 

KeyboardInterrupt: 

In [ ]:
opening_gpt.randomize_all_summaries(print_debug=True)